## Preprocessing данных для подготовки финального датасета

In [69]:
import os 
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

In [70]:
path = './_data.csv'
df = pd.read_csv(path)

In [71]:
# Исключение адресов вне Москвы.
df = df[df["Адрес"].str.contains("Москва")]
# Исключение колонок, которые были признаны подлежащими исключению по результатам EDA и с учётом дополнительного обсуждения внутри команды.
to_drop = ['Unnamed: 0', 'Тип', 'Адрес', 'Телефоны', 'Описание', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 
           'Дополнительно', 'Название ЖК', 'Парковка', 'Серия дома', 'Высота потолков, м', 'Мусоропровод', 'Ссылка на объявление', 'Лифт']
df.drop(columns=to_drop, inplace=True)

In [72]:
# Работа с колонкой "Количество комнат": получение значения общего числа комнат в допустимом формате.
df['rooms'] = df['Количество комнат'].apply(lambda x: int(x.split(',')[0]) if pd.notnull(x) else x)
df.drop(columns=['Количество комнат'], inplace=True)
# Ввиду небольшого числа пропусков, команда сочла допустимым использование метода Complete Case Analysis с удалением строк с отсутствующими значениями.
df.dropna(subset=['rooms'], inplace=True)

In [73]:
# Работа с колонкой "Метро": получение наименования станций. Ввиду ресурсных ограничений и непроработанности гипотез, информация о времени перемещения до станции метро была исключена. 
df['stations'] = df['Метро'].apply(lambda x: x.split(' ')[1] if pd.notnull(x) else x)
df.drop(columns=['Метро'], inplace=True)
# Ввиду небольшого числа пропусков и учитывая содержание соответствующих строк, команда сочла допустимым использование метода Complete Case Analysis с удалением строк с отсутствующими значениями.
# Более качественным и устойчивым к искажениям решением было бы кодирование непосредственно адресов квартир (подлежит дополнительному анализу). 
df.dropna(subset=['stations'], inplace=True)
# На последующем этапе предполагается выполнение OHE колонки 'stations'.

In [74]:
# Работа с колонкой "Площадь, м2": получение значения общего числа комнат в допустимом формате.
df['area'] = df['Площадь, м2'].apply(lambda x: float(x.split('/')[0]))
df.drop(columns=['Площадь, м2'], inplace=True)
# Команда не пришла к однозначному выводу о необходимости устранения выбросов (квартиры большой метражности) ввиду рисков утраты ценной информации.

In [75]:
# Работа с колонкой "Дом": получение значений этажности домов, этажей расположения квартир в домах. Ввиду значительного числа пропусков, значения типов домов исключены из финального датасета.
df['house'] = df['Дом'].apply(lambda x: x.split(', '))
df['floors'] = df.loc[:,'house'].apply(lambda x: x[0])
df['apartment_floors'] = df.loc[:,'floors'].apply(lambda x: int(x.split('/')[0]))
df['total_floors'] = df.loc[:,'floors'].apply(lambda x: int(x.split('/')[1]))
df.drop(columns=['floors', 'house', 'Дом'], inplace=True)
# Осуществить OHE

In [76]:
# Работа с колонкой "Цена": получение значений ежемесячной стоимости аренды в требуемом формате. Ввиду отсутствия качественных гипотез о стоимости коммунальных услуг, данный параметр не был сохранён в датасете. 
df['price'] = df['Цена'].apply(lambda x: float(x.split()[0]))
df.drop(columns=['Цена'], inplace=True)

In [77]:
# Работа с колонкой "Ремонт": принимая во внимание явную градацию видов ремонтов по стоимости, принято решение заменить значения по шкале от 0 (Без ремонта) до 3 (Дизайнерский). 
# Команда исходила из предположения, что отсутствующие значения, вероятно, соответствуют квартирам, в которых не был произведён ремонт. 
df['Ремонт'] = df['Ремонт'].fillna(0)
dict_repairs = {0: 0, 'Без ремонта': 0, 'Косметический': 1, 'Евроремонт': 2,  'Дизайнерский': 3}
df['repairs'] = df['Ремонт'].apply(lambda x: dict_repairs[x])
df.drop(columns='Ремонт', inplace=True)

In [78]:
# Работа с колонкой "Можно с детьми/животными": создание двух колонок (children, animals). 
# Данное решение принято исходя из логики эксплицитного указания в объявлениях на разрешение проживания детей или животных в квартире (презюмируется запрет). 
# Соответственно, отсутствующие значения интерпретированы как запрет.
df['Можно с детьми/животными'].head(15)
df['temp_children_animals'] = df['Можно с детьми/животными'].apply(lambda x: x.split(', ') if pd.notnull(x) else [0])
df['children'] = df['temp_children_animals'].apply(lambda x: 1 if 'Можно с детьми' in x else 0)
df['animals'] = df['temp_children_animals'].apply(lambda x: 1 if 'Можно с животными' in x else 0)
df.drop(columns=['Можно с детьми/животными', 'temp_children_animals'], inplace=True)

In [79]:
# Замена названий колонок на латиницу.
df.rename(columns={"ID  объявления": "ID"}, inplace=True)
df.to_csv('./data.csv', index=False)  
df.isnull().sum()
df.dtypes

ID                    int64
rooms               float64
stations             object
area                float64
apartment_floors      int64
total_floors          int64
price               float64
repairs               int64
children              int64
animals               int64
dtype: object